## 01) Análise literária: 
- Definição dos códigos para as pesquisas literárias, uso dos módulos: Bio e habanero.

In [1]:
pip install Bio

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Função que procura o termo de interesse na database de interesse:
from Bio import Entrez
Entrez.email = 'A.N.Other@example.com'
def buscar_artigos(term, max_artigos):
    """
    Esta função tem como input o termo de busca e como output devolve uma lista de IDs dos artigos encontrados e o nº de artigos encontrados
    """
    links = Entrez.esearch(db="pubmed", retmax = max_artigos, term=term)	
    l = Entrez.read(links)
    ids = l.get("IdList")
    return {"Nº de artigos encontrados": l.get("Count"), "ID dos artigos": ids}


In [3]:
pip install habanero

Note: you may need to restart the kernel to use updated packages.


In [7]:
# Função auxiliar para determinar o URL do artigo:
from habanero import Crossref 
def determinar_url(doi):
  cr = Crossref()
  article = cr.works(ids="10.1101/2023.11.20.567899").get("message")
  return article['URL']

In [9]:
# Função que retorna o título do artigo, data de publicação, nome dos autores e o URL do documento
from Bio import Medline
def dados_artigos(ids_art):
    """
    O input é o ID do artigo de interesse e retorna o título do artigo, data de publicação, nome dos autores e o URL do documento
    """
    handle = Entrez.efetch(db="pubmed", id=ids_art, rettype="medline", retmode="text")
    lista = Medline.parse(handle)

    for r in lista:
        print("------------------")
        print("Título do Artigo:", r.get("TI"))
        print("Data de publicação:", r.get("DP"))
        print("Autores:", r.get("AU"))
        doi = r.get("LID")

        for x,y in enumerate(doi.split()):
           if y == "[doi]":
            print("URL:", determinar_url(doi.split()[x-1]))
        print("------------------")
    handle.close()

#### *Resultados da busca literária para os genes de interesse:*
- Genes de busca: IL13, STAT6, IL10 e TGFB1.
- A busca foi realizada para 20 artigos para cada gene de interesse.

<span style='color: red;'>Os resultados alteram a partir da data de pesquisa.</span>

In [4]:
#Termos de pesquisa:
terms = ["IL13", "STAT6", "IL10", "TGFB1"]

In [10]:
# Resultados das pesquisa de no máximo 20 artigos para cada termo:
term1 = buscar_artigos(terms[0], 20)
term2 = buscar_artigos(terms[1], 20)
term3 = buscar_artigos(terms[2], 20)
term4 = buscar_artigos(terms[3], 20)

print("Pesquisa para o gene {} \n".format(terms[0]), term1, "\n","Pesquisa para o gene {} \n".format(terms[1]),term2, "\n","Pesquisa para o gene {} \n".format(terms[2]),term3, "\n","Pesquisa para o gene {} \n".format(terms[3]),term4)

Pesquisa para o gene IL13 
 {'Nº de artigos encontrados': '17278', 'ID dos artigos': ['38154627', '38152869', '38150981', '38148813', '38147892', '38145912', '38145351', '38143940', '38142785', '38142780', '38141862', '38141197', '38140869', '38140264', '38137445', '38135676', '38130766', '38127798', '38125743', '38124753']} 
 Pesquisa para o gene STAT6 
 {'Nº de artigos encontrados': '4311', 'ID dos artigos': ['38144091', '38136901', '38136399', '38134634', '38133747', '38114221', '38112587', '38104875', '38098120', '38095318', '38087324', '38082280', '38079450', '38070062', '38069015', '38062486', '38054829', '38044939', '38026128', '38019126']} 
 Pesquisa para o gene IL10 
 {'Nº de artigos encontrados': '82674', 'ID dos artigos': ['38156574', '38156384', '38155972', '38155134', '38154763', '38154762', '38154490', '38154347', '38153856', '38153510', '38153396', '38153198', '38152404', '38152266', '38151179', '38150908', '38150744', '38149388', '38149385', '38149209']} 
 Pesquisa para

In [11]:
# Artigos para o gene: IL13
dados_artigos(term1.get("Ids dos artigos definidos"))

HTTPError: HTTP Error 400: Bad Request

In [ ]:
# Artigos para o gene: STAT6
dados_artigos(term2.get("Ids dos artigos definidos"))

In [ ]:
# Artigos para o gene: IL10
dados_artigos(term3.get("Ids dos artigos definidos"))

In [ ]:
# Artigos para o gene: TGFB1
dados_artigos(term4.get("Ids dos artigos definidos"))

## 02) Análise da sequência e das features presentes no NCBI
- Dowload dos genes e criação de documentos com as informações obtidas.
- Os accesion dos genes:
  - TGFB1 : NG_013364
  - IL13  : NG_012090
  - IL10  : NG_012088
  - STAT6 : NG_021272

In [8]:
# Função para o dowload do gene
from Bio import Entrez
from Bio import SeqIO
Entrez.email = 'A.N.Other@example.com'
def dowload_gene(id_gene : str, name_gene : str):
    handle = Entrez.efetch(db="nucleotide", rettype="gb", retmode="text",id= id_gene)
    for seq_record in SeqIO.parse(handle, "gb"):
        SeqIO.write(seq_record, "{}.gb".format(name_gene), "gb")

In [ ]:
# Dowload dos genes de interesse:
dowload_gene("NG_013364", "TGFB1")
dowload_gene("NG_012090", "IL13")
dowload_gene("NG_012088", "IL10")
dowload_gene("NG_021272", "STAT6")

In [1]:
#Função para fazer download do ficheiro FASTA com as sequências dos genes
from Bio import Entrez
from Bio import SeqIO
Entrez.email = 'A.N.Other@example.com'
def download_fasta(gene_id, filename):
    try:
        handle = Entrez.efetch(db="nucleotide", id=gene_id, rettype="fasta", retmode="text")
        with open(filename, "w") as out_file:
            out_file.write(handle.read())
        handle.close()
        print(f"Download do FASTA do gene {gene_id} completado.")
    except Exception as e:
        print(f"Erro: {e}")


In [2]:
download_fasta('NG_012090', 'IL13.fasta')

Download do FASTA do gene NG_012090 completado.


In [3]:
download_fasta('NG_012088', 'IL10.fasta')
download_fasta('NG_021272', 'STAT6.fasta')
download_fasta('NG_013364', 'TGFB1.fasta')

Download do FASTA do gene NG_012088 completado.
Download do FASTA do gene NG_021272 completado.
Download do FASTA do gene NG_013364 completado.
